In [1]:
# !pip install ensemble-boxes

In [2]:
import pandas as pd
import os
from glob import glob
from ensemble_boxes import *
import numpy as np

In [3]:
# 앙상블 폴더 생성
os.makedirs('/opt/ml/code/Swin-Transformer-Object-Detection/submissions', exist_ok=True)
os.makedirs('/opt/ml/code/Swin-Transformer-Object-Detection/submissions/before_ensemble', exist_ok=True)

# 테스트 데이터 이미지 아이디
image_ids = np.load('/opt/ml/code/Swin-Transformer-Object-Detection/image_ids.npy',allow_pickle = True)

# csv 경로 모으기
csv_paths = '/opt/ml/code/Swin-Transformer-Object-Detection/submissions/before_ensemble'
csv_list = glob(csv_paths+"/*")

# 데이터프레임 모으기
dataframes = []
for csv in csv_list:
    dataframes.append(pd.read_csv(csv))


In [4]:
# 이미지 1장의 바운딩 박스 문자열 Parse
def get_data(pred:str):
    
    boxes = []
    scores = []
    labels = []
    
    pred = pred.strip()
    pred = pred.split(" ")
    # label, score, x1, y1, x2, y2
    for i,start in enumerate(range(0,len(pred),6)):
        boxes.append(list(map(float,pred[start+2:start+6])))
        scores.append(float(pred[start+1]))
        labels.append(int(pred[start]))
        
    boxes = np.array(boxes) # [[x1,y1,x2,y2],[x1,y1,x2,y2],...]
    scores = np.array(scores) # [s1,s2,...]
    labels = np.array(labels) # [l1,l2,...]
    
    return boxes, scores, labels

def make_prediction(boxes,scores,labels):
    
    """
     boxes, scores, labels: arrays
    """
    
    wbf_pred_str = ''
    for box, score, label in zip(boxes,scores,labels):
        wbf_pred_str += str(label) + ' '+ str(score) + ' ' + str(box[0]) + ' ' + \
                        str(box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
    
    return wbf_pred_str

In [5]:
# skip_box_thr = 0.7
skip_box_thr = 0.01
iou_thr = 0.6
# weights = [5,2,3,1] 

predictions = []
file_names = []

# 이미지 순회
for i,image_id in enumerate(image_ids):
    boxes_list = []
    scores_list = []
    labels_list = []
    
    # 모델 순회
    for df in dataframes:        
        pred = df['PredictionString'].iloc[i]
        # arrays
        boxes, scores, labels = get_data(pred) 

        # weighted_boxes_fusion은 [0,1]로 정규화된 좌표값을 받음
        boxes = boxes/512.
        boxes_list.append(boxes)
        scores_list.append(scores)
        labels_list.append(labels)

    boxes, scores, labels = weighted_boxes_fusion(boxes_list,scores_list,labels_list,iou_thr=iou_thr,skip_box_thr=skip_box_thr)
    boxes = boxes*512
    wbf_pred_str = make_prediction(boxes,scores,labels)
    predictions.append(wbf_pred_str)


In [6]:
wbf_df = pd.DataFrame()
wbf_df['PredictionString'] = predictions
wbf_df['image_id'] = image_ids

csv_name = 'wbf_kyubin.csv'

save_dir = '/opt/ml/code/Swin-Transformer-Object-Detection/submissions'

wbf_df.to_csv(os.path.join(save_dir, csv_name), index=None)
wbf_df.head()

,PredictionString,image_id
0,8 0.9754363 173.20209 235.67116 463.73138 472....,batch_01_vt/0021.jpg
1,8 0.977294921875 159.12901306152344 58.1549072...,batch_01_vt/0028.jpg
2,1 0.9798071384429932 71.68167114257812 169.063...,batch_01_vt/0031.jpg
3,8 0.9067034721374512 102.75265502929688 122.10...,batch_01_vt/0032.jpg
4,8 0.993256688117981 150.30381774902344 0.27757...,batch_01_vt/0070.jpg
